# This is an implementation of an inverse kinematics algorithm.

In [1]:
import sys
import os

if 'google.colab' in sys.modules:
  print("You are in colab")
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/Othercomputers/Mi portátil/gcs-motion-planning/jupyter_notebooks')
  !pip install pyngrok drake
else:
  os.chdir(os.path.dirname(os.getcwd()))
  print('You are running local.')

import numpy as np

from pyngrok import ngrok

# Drake Libraries
# Libraries for basic sdf file visualization
from pydrake.common import temp_directory
from pydrake.multibody.plant import MultibodyPlant
from pydrake.systems.analysis import Simulator
from pydrake.multibody.parsing import Parser
import random

from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Cylinder,
    DiagramBuilder,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    RigidTransform,
    Role,
    RollPitchYaw,
    RotationMatrix,
    Solve,
    StartMeshcat,
)

# add parent folders to path
sys.path.append(os.path.abspath('..'))


You are in colab
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 MB 7.5 MB/s eta 0:00:00


In [2]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

print(os.getcwd())
base_dir = os.path.dirname(os.getcwd())
print(base_dir)


INFO:drake:Meshcat listening for connections at http://localhost:7000


Meshcat is accessible at: NgrokTunnel: "https://cd5f-35-245-181-238.ngrok-free.app" -> "http://localhost:7000"
/content/drive/Othercomputers/Mi portátil/gcs-motion-planning/jupyter_notebooks
/content/drive/Othercomputers/Mi portátil/gcs-motion-planning


In [3]:
def add_custom_robot(plant,base_dir):
    """
    Adds a custom robot to the plant from an SDF file.

    Args:
        plant: The MultibodyPlant to which the robot will be added.
        sdf_path: The path to the SDF file of the robot.

    Returns:
        The model instance of the added robot.
    """
    sdf_path= os.path.join(base_dir,'common-files','models','KINOVA_GEN3_LITE_PRIMITIVES','kinova_model.sdf')

    if not os.path.exists(sdf_path):
        raise FileNotFoundError(f"SDF file not found at {sdf_path}")

    parser = Parser(plant)
    robot_model = parser.AddModels(sdf_path)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("BASE"))
    return robot_model[0]

def add_table(plant,base_dir):
    """
    Adds a table to the plant from an SDF file.

    Args:
        plant: The MultibodyPlant to which the table will be added.
        table_path: The path to the SDF file of the table.

    Returns:
        The model instance of the added table.
    """

    table_path = os.path.join(base_dir,'common-files','scene_objects','table','table_wide.sdf')

    if not os.path.exists(table_path):
        raise FileNotFoundError(f"SDF file not found at {table_path}")
    parser = Parser(plant)
    table_model = parser.AddModels(table_path)
    translation_vector = RigidTransform([0.5, -0.5, 0.0])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table_body"),translation_vector)
    return table_model

## Run IRIS on manually-specified seeds

In [4]:
def create_scene_and_multibody_plant(plant, builder):
    """
    Adds models to the plant

    Returns:
      A tuple of the instances added to the plant.
    """
    robot_model = add_custom_robot(plant)[0]
    table_model = add_table(plant)[0]
    plant.Finalize()

    return (robot_model, table_model)


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

robot = add_custom_robot(plant,base_dir)
table = add_table(plant,base_dir)
plant.Finalize()

In [6]:
visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(delete_prefix_initialization_event=False),
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
q0 = plant.GetPositions(plant_context)

diagram.ForcedPublish(context)

In [7]:
meshcat.Delete()
meshcat.DeleteAddedControls()

In [ ]:
"""
Desired Position = [0.5,0.0,0.5]
Desired Rotation matrix = [0, 0, 1;
                           0, 1, 0
                           -1,0, 0];
"""

In [ ]:
def apply_inverse_kinematics(plant, plant_context, pose : list = [[0.5, 0.2, 0.5]]):

    from pydrake.multibody.inverse_kinematics import InverseKinematics
    from pydrake.math import RigidTransform
    import numpy as np
    from pydrake.solvers import Solve

    desired_pose = RigidTransform(pose)

    # Create an inverse kinematics instance
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Joint variables to solve for

    # Get the end-effector frame
    end_effector_frame = plant.GetFrameByName("tool_center_point")  # Adjust as needed

    # Define the exact position constraint (ensure (3,1) shape)
    lower_bound = np.array(pose) - np.array([-0.2, -0.2, -0.2])
    upper_bound = np.array(pose) + np.array([0.2, 0.2, 0.2])
    p_BQ = np.array([0.0, 0.0, 0.0])  # The constrained point in the end-effector frame

    # ✅ Use `AddPositionConstraint()`
    ik.AddPositionConstraint(
        frameB=end_effector_frame,
        p_BQ=p_BQ,
        frameA=plant.world_frame(),
        p_AQ_lower=lower_bound,
        p_AQ_upper=upper_bound
    )
    q0 = np.array([0.0, 0.6981, 0.0, 1.5708, 0.0, 0.0])  # Adjust to your robot's nominal position
    W = np.eye(len(q0))
    prog = ik.get_mutable_prog()
    prog.AddQuadraticErrorCost(np.identity(len(q_variables))*10, q0, q_variables)
    prog.SetInitialGuess(q_variables, q0)

    # Solve for joint positions
    result = Solve(ik.prog())

    if result.is_success():
        #q_solution = result.GetSolution(q_variables)
        #print("Solved joint positions:", q_solution)
        return q_solution
    else:
        print("IK problem failed to find a solution.")
        return None


In [ ]:
import time
import math

pose = [0.25, 0.0, 0.5]
for i in range(100):  # Iterate multiple times for continuous update
    q_solution = apply_inverse_kinematics(plant=plant, plant_context=plant_context, pose=pose)
    print("pose",pose)
    pose[0] = 0.25*math.sin(0.1*i)+0.25
    time.sleep(0.1)
    diagram.ForcedPublish(context)
    if q_solution is not None:
      plant.SetPositions(plant_context, q_solution)